<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/IOU%E6%A4%9C%E8%A8%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# for Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!mkdir input output

In [ ]:
!kaggle competitions download -c foursquare-location-matching
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

 96% 153M/159M [00:02<00:00, 77.8MB/s]
100% 159M/159M [00:02<00:00, 73.6MB/s]
Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [ ]:
!pip install Levenshtein
!pip install pykakasi
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 258 kB 13.3 MB/s 
     |████████████████████████████████| 1.8 MB 60.7 MB/s 
     |████████████████████████████████| 103 kB 52.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 13.5 MB/s 
  Created wheel for jaconv: filename=jaconv-0.3-py3-none-any.whl size=15564 sha256=b2899fa640e0211e571fc4ac3b3345eba508588485b463c314934ee4bc0ebf9c
  Stored in directory: /root/.cache/pip/wheels/8f/4f/c2/a2a3b14d0e94f855f4aa8887bf0267bee9ecfb8e62a9ee2d92
Successfully built jaconv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# change directory

In [ ]:
%cd /content/output

/content/output


# start code

In [ ]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import time
import random
import Levenshtein
import difflib
import multiprocessing
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import pykakasi
from tqdm.notebook import tqdm
import math

from fuzzywuzzy import fuzz


tqdm.pandas()

## Conversion kanji to alphabet

In [ ]:
def convert_japanese_alphabet(df: pd.DataFrame):
    kakasi = pykakasi.kakasi()
    kakasi.setMode('H', 'a')  # Convert Hiragana into alphabet
    kakasi.setMode('K', 'a')  # Convert Katakana into alphabet
    kakasi.setMode('J', 'a')  # Convert Kanji into alphabet
    conversion = kakasi.getConverter()

    def convert(row):
        for column in ["name", "address", "city", "state"]:
            try:
                row[column] = conversion.do(row[column])
            except:
                pass
        return row

    df[df["country"] == "JP"] = df[df["country"] == "JP"].progress_apply(convert, axis=1)
    return df

## start 

In [ ]:
## Parameters
is_debug = False
SEED = 2022
num_neighbors = 3
num_neighbors2 = 40
num_split = 10
# feat_columns = ['name', 'address', 'city', 
#             'state', 'zip', 'url', 
#            'phone', 'categories', 'country']
feat_columns = ['dist', 'name', 'address', 'city', 
            'state', 'zip', 'url', 
           'phone', 'categories', 'country']
vec_columns = ['name', 'categories', 'address', 
               'state', 'url', 'country']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(SEED)

In [ ]:
%load_ext Cython

In [ ]:
%%cython
def LCS(str S, str T):
    cdef int i, j
    cdef list dp = [[0] * (len(T) + 1) for _ in range(len(S) + 1)]
    for i in range(len(S)):
        for j in range(len(T)):
            dp[i + 1][j + 1] = max(dp[i][j] + (S[i] == T[j]), dp[i + 1][j], dp[i][j + 1], dp[i + 1][j + 1])
    return dp[len(S)][len(T)]

In [ ]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

# get manhattan distance
def manhattan(lat1, long1, lat2, long2):
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)

# get haversine distance
def vectorized_haversine(lats1, lats2, longs1, longs2):
    radius = 6371
    dlat=np.radians(lats2 - lats1)
    dlon=np.radians(longs2 - longs1)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lats1)) \
        * np.cos(np.radians(lats2)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = radius * c
    return d

In [ ]:
def recall_simple(df):
    threshold = 2
    
    val2id_d = {}
    for col in rec_columns:
        temp_df = df[['id', col]]
        temp_df[col] = temp_df[col].str.lower()
        val2id = temp_df.groupby(col)['id'].apply(set).to_dict()
        val2id_d[col] = val2id
        del val2id
    
    cus_ids = []
    match_ids = []
    for vals in tqdm(df[rec_columns + ['id']].fillna('null').values):
        cus_id = vals[-1]
        match_id = []
        
        rec_match_count = []
        for i in range(len(rec_columns)):
            col = rec_columns[i]
            
            if vals[i] != 'null':
                rec_match_count += list(val2id_d[col][vals[i].lower()])
        rec_match_count = dict(Counter(rec_match_count))
        
        for k, v in rec_match_count.items():
            if v > threshold:
                match_id.append(k)
        
        cus_ids += [cus_id] * len(match_id)
        match_ids += match_id
    
    train_df = pd.DataFrame()
    train_df['id'] = cus_ids
    train_df['match_id'] = match_ids
    train_df = train_df.drop_duplicates()
    del cus_ids, match_ids
    
    num_data = len(train_df)
    num_data_per_id = num_data / train_df['id'].nunique()
    print('Num of data: %s' % num_data)
    print('Num of data per id: %s' % num_data_per_id)
    
    return train_df

In [ ]:
def recall_knn(df, Neighbors = 10, Neighbors2 = 10 ):
    print('Start knn grouped by country')
    train_df_country = []
    for country, country_df in tqdm(df.groupby('country')):
        country_df = country_df.reset_index(drop = True)

        neighbors = min(len(country_df), Neighbors)
        knn = KNeighborsRegressor(n_neighbors = neighbors,
                                    metric = 'haversine',
                                    n_jobs = -1)
        knn.fit(country_df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180),
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']].apply(lambda dr : (dr*math.pi)/180), 
            return_distance = True)
        knn.fit(country_df[['latitude','longitude']],
                country_df.index)
        dists, nears = knn.kneighbors(
            country_df[['latitude', 'longitude']], 
            return_distance = True)

        for k in range(neighbors):            
            cur_df = country_df[['id']]
            cur_df['match_id'] = country_df['id'].values[nears[:, k]]
            cur_df['kdist_country'] = dists[:, k]
            cur_df['kneighbors_country'] = k
            
            train_df_country.append(cur_df)
    train_df_country = pd.concat(train_df_country)
    
    print('Start knn')
    train_df = []
    knn = NearestNeighbors(n_neighbors = Neighbors2)
    knn.fit(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180), df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']].apply(lambda dr : (dr*math.pi)/180))
    knn.fit(df[['latitude','longitude']], df.index)
    dists, nears = knn.kneighbors(df[['latitude','longitude']])
    
    for k in range(Neighbors2):            
        cur_df = df[['id']]
        cur_df['match_id'] = df['id'].values[nears[:, k]]
        cur_df['kdist'] = dists[:, k]
        cur_df['kneighbors'] = k
        train_df.append(cur_df)
    
    train_df = pd.concat(train_df)
    train_df = train_df.merge(train_df_country,
                                 on = ['id', 'match_id'],
                                 how = 'outer')
    del train_df_country
    
    return train_df

In [ ]:
def add_features(df):    
    for col in tqdm(feat_columns):
        if col == 'dist':
            lat = data.loc[df['id']]['latitude'].values
            match_lat = data.loc[df['match_id']]['latitude'].values
            lon = data.loc[df['id']]['longitude'].values
            match_lon = data.loc[df['match_id']]['longitude'].values
            df['latdiff'] = (lat - match_lat)
            df['londiff'] = (lon - match_lon)
            df['manhattan'] = manhattan(lat, lon, match_lat, match_lon)
            df['euclidean'] = (df['latdiff'] ** 2 + df['londiff'] ** 2) ** 0.5
            df['haversine'] = vectorized_haversine(lat, match_lat, lon, match_lon)
            continue
        
        col_values = data.loc[df['id']][col].values.astype(str)
        matcol_values = data.loc[df['match_id']][col].values.astype(str)
        
        if col in vec_columns:
            tv_fit = tfidf_d[col]
            indexs = [id2index_d[i] for i in df['id']]
            match_indexs = [id2index_d[i] for i in df['match_id']]                    
            df[f'{col}_sim'] = tv_fit[indexs].multiply(tv_fit[match_indexs]).\
                                            sum(axis = 1).A.ravel()
        
        geshs = []
        levens = []
        jaros = []
        lcss = []
        fuzwuz = []
        for s, match_s in zip(col_values, matcol_values):
            if s != 'nan' and match_s != 'nan':                    
                geshs.append(difflib.SequenceMatcher(None, s, match_s).ratio())
                levens.append(Levenshtein.distance(s, match_s))
                jaros.append(Levenshtein.jaro_winkler(s, match_s))
                lcss.append(LCS(str(s), str(match_s)))
                fuzwuz.append(fuzz.ratio(s, match_s))
            else:
                geshs.append(np.nan)
                levens.append(np.nan)
                jaros.append(np.nan)
                lcss.append(np.nan)
                fuzwuz.append(np.nan)
        
        df[f'{col}_gesh'] = geshs
        df[f'{col}_leven'] = levens
        df[f'{col}_jaro'] = jaros
        df[f'{col}_lcs'] = lcss
        df[f'{col}_fuz'] = fuzwuz
        
        if col not in ['phone', 'zip']:
            df[f'{col}_len'] = list(map(len, col_values))
            df[f'match_{col}_len'] = list(map(len, matcol_values)) 
            df[f'{col}_len_diff'] = np.abs(df[f'{col}_len'] - df[f'match_{col}_len']) /\
                                        df[f'{col}_len'] 
            df[f'{col}_nleven'] = df[f'{col}_leven'] / \
                                    df[[f'{col}_len', f'match_{col}_len']].max(axis = 1)
            
            df[f'{col}_nlcsk'] = df[f'{col}_lcs'] / df[f'match_{col}_len']
            df[f'{col}_nlcs'] = df[f'{col}_lcs'] / df[f'{col}_len']
            
            df = df.drop(f'{col}_len', axis = 1)
            df = df.drop(f'match_{col}_len', axis = 1)
            gc.collect()
            
    return df

In [ ]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [ ]:
## Data load
data_root = '../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if is_debug:
    data = data.sample(n = 10000, random_state = SEED)
    data = data.reset_index(drop = True)

In [ ]:
data[data["country"] == "JP"].head(2)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
12,E_0000c566a81ea1,つじ田 味噌の章,35.694796,139.767348,神田小川町1-1,千代田区,東京都,101-0052,JP,https://tsukemen-tsujita.com,366597676,Ramen Restaurants,P_5fcd8cd98fea2b
17,E_00014f107dc217,ローソン 弘前撫牛子店,40.623279,140.494751,撫牛子2-10ｰ2,弘前市,青森県,036-8075,JP,http://www.lawson.co.jp,172325353,Convenience Stores,P_d0bb3bdad1ab4f


In [ ]:
data = convert_japanese_alphabet(data)

  0%|          | 0/70032 [00:00<?, ?it/s]

In [ ]:
data[data["country"] == "JP"].head(2)

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
12,E_0000c566a81ea1,tsujita misonoshou,35.694796,139.767348,kandaogawachou1-1,chiyodaku,toukyouto,101-0052,JP,https://tsukemen-tsujita.com,366597676,Ramen Restaurants,P_5fcd8cd98fea2b
17,E_00014f107dc217,rooson hirosakibuushikomise,40.623279,140.494751,buushiko2-10ｰ2,hirosakishi,aomoriken,036-8075,JP,http://www.lawson.co.jp,172325353,Convenience Stores,P_d0bb3bdad1ab4f


In [ ]:
## Data split
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(data, 
                                                data['point_of_interest'], 
                                                data['point_of_interest'])):
    data.loc[val_idx, 'set'] = 1-i  # 1-i 逆にした

print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

valid_data = data[data['set'] == 0]
train_data = data[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Valid data: ')
analysis(valid_data)

train_poi = train_data['point_of_interest'].unique().tolist()
valid_poi = valid_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(valid_poi))

train_ids = train_data['id'].unique().tolist()
valid_ids = valid_data['id'].unique().tolist()
      
print(set(train_ids) & set(valid_ids))
      
tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['valid_ids'] = valid_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, valid_data
gc.collect()

data = data.set_index('id')
data = data.loc[tv_ids_d['train_ids']]
data = data.reset_index()

Num of train data: 1138812
0.0    569406
1.0    569406
Name: set, dtype: int64
Train data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369985
Mean num of unique poi: 1.5389975269267673
Valid data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369987
Mean num of unique poi: 1.5389892077289202
set()
set()


In [ ]:
len(data)

569406

In [ ]:
%%time
## Train data generated by knn
id2index_d = dict(zip(data['id'].values, data.index))

tfidf_d = {}
for col in vec_columns:
    tfidf = TfidfVectorizer()
    tv_fit = tfidf.fit_transform(data[col].fillna('nan'))
    tfidf_d[col] = tv_fit

train_data = recall_knn(data, num_neighbors, num_neighbors2)

data = data.set_index('id')
ids = train_data['id'].tolist()
match_ids = train_data['match_id'].tolist()

poi = data.loc[ids]['point_of_interest'].values
match_poi = data.loc[match_ids]['point_of_interest'].values

train_data['label'] = np.array(poi == match_poi, dtype = np.int8)
del poi, match_poi, ids, match_ids
gc.collect()

print('Num of unique id: %s' % train_data['id'].nunique())
print('Num of train data: %s' % len(train_data))
print('Pos rate: %s' % train_data['label'].mean())
print("rate = ", len(train_data)/13928057)
print(train_data.sample(5))

Start knn grouped by country


  0%|          | 0/209 [00:00<?, ?it/s]

Start knn
Num of unique id: 569406
Num of train data: 22849117
Pos rate: 0.04618664257354015
rate =  1.6405100151442515
                        id          match_id     kdist  kneighbors  \
3971940   E_f9ba2162f95dcd  E_662400acae66fa  0.014700         6.0   
6272554   E_04245bda18eb11  E_fb07dc726d84d8  0.014393        11.0   
10123508  E_c73eb508149757  E_40b01040ba5cc5  0.006533        17.0   
7148274   E_8e033c1dd4b937  E_cdc56b3d702678  0.002664        12.0   
8896276   E_9fb153bdc196e1  E_a92aa0a94248ab  0.002723        15.0   

          kdist_country  kneighbors_country  label  
3971940             NaN                 NaN      0  
6272554             NaN                 NaN      0  
10123508            NaN                 NaN      0  
7148274             NaN                 NaN      0  
8896276             NaN                 NaN      0  
CPU times: user 13min 21s, sys: 9.49 s, total: 13min 31s
Wall time: 6min 45s


In [ ]:
sum(train_data['kdist'].isna()), sum(train_data['kdist'].notna()),  len(train_data)

(72877, 22776240, 22849117)

## 6/18 追加（反映チェック必要） kdistに距離を集約

In [ ]:
# pos = train_data['kdist_country'].notna()
# train_data.loc[pos, 'kdist'] = train_data[pos]['kdist_country']

In [ ]:
sum(train_data['kdist'].isna())

0

In [ ]:
train_data[pos]

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label
0,E_000002eae2a589,E_000002eae2a589,0.000000,0.0,0.000000,0.0,1
1,E_000007f24ebc95,E_000007f24ebc95,0.000000,0.0,0.000000,0.0,1
2,E_000008a8ba4f48,E_000008a8ba4f48,0.000000,0.0,0.000000,0.0,1
3,E_00001d92066153,E_00001d92066153,0.000000,0.0,0.000000,0.0,1
4,E_000023d8f4be44,E_000023d8f4be44,0.000000,0.0,0.000000,0.0,1
...,...,...,...,...,...,...,...
22849112,E_eda20d05aeff71,E_d2207973e63e4f,0.168357,NaN,0.168357,2.0,0
22849113,E_f0529eb8c2693c,E_fac45dafb5ea5f,0.068799,NaN,0.068799,2.0,0
22849114,E_f792bc6a64a1be,E_d0e5d236fb2973,0.012524,NaN,0.012524,2.0,0
22849115,E_fac45dafb5ea5f,E_f0529eb8c2693c,0.068799,NaN,0.068799,2.0,0


In [ ]:
train_data[train_data['kdist'].isna()]

,id,match_id,kdist,kneighbors,kdist_country,kneighbors_country,label


In [ ]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

eval_df = pd.DataFrame()
eval_df['id'] = data['id'].unique().tolist()
eval_df['match_id'] = eval_df['id']
print('Unique id: %s' % len(eval_df))

eval_df_ = train_data[train_data['label'] == 1][['id', 'match_id']]
eval_df = pd.concat([eval_df, eval_df_])

eval_df = eval_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
eval_df['matches'] = eval_df['match_id'].apply(lambda x: ' '.join(set(x)))
print('Unique id: %s' % len(eval_df))

iou_score = get_score(eval_df)
print('IoU score: %s' % iou_score)

Unique id: 569406
Unique id: 569406
IoU score: 0.9375596682994936


In [ ]:
print('IoU score: %s' % iou_score, num_neighbors, num_neighbors2)

IoU score: 0.9375596682994936 3 40


In [ ]:
# IoU score: 0.9210875384608563 

In [ ]:
#IoU score: 0.936960291817873 1,39
#IoU score: 0.9344528470303082 5 35
#IoU score: 0.9375555341479278 3 40
#IoU score: 0.9375531925264253 2 40
#IoU score: 0.9375520217156739 1 40 1.6352776270229221
#IoU score: 0.9375555341479278 3 40 1.6353082127679404
#----------->  not radian #IoU score: 0.9375596682994936 3 40 1.6405100151442515

In [ ]:
## Add features
count = 0
start_row = 0

data = data.set_index('id')
unique_id = train_data['id'].unique().tolist()
num_split_id = len(unique_id) // num_split
for k in range(1, num_split + 1):
    print('Current split: %s' % k)
    end_row = start_row + num_split_id
    if k < num_split:
        cur_id = unique_id[start_row : end_row]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    else:
        cur_id = unique_id[start_row: ]
        cur_data = train_data[train_data['id'].isin(cur_id)]
    
    cur_data = add_features(cur_data)
    print(cur_data.shape)
    print(cur_data.sample(1))
    
    cur_data.to_csv('train_data%s.csv' % k, index = False)    
    print(cur_data.columns)
    start_row = end_row
    count += len(cur_data)
    
    del cur_data
    gc.collect()
    
print(count)

Current split: 1


  0%|          | 0/10 [00:00<?, ?it/s]

(2284691, 91)
                        id          match_id     kdist  kneighbors  \
19964640  E_0ff5d8fd20216b  E_9b2fd9a0a2312f  0.030003        35.0   

          kdist_country  kneighbors_country  label  latdiff   londiff  \
19964640            NaN                 NaN      0  0.01991  0.022444   

          manhattan  ...  country_sim  country_gesh  country_leven  \
19964640   0.042355  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
19964640           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
19964640             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nlev

  0%|          | 0/10 [00:00<?, ?it/s]

(2284897, 91)
                        id          match_id     kdist  kneighbors  \
16046371  E_2e40fe80129784  E_339b7e7488f126  0.000877        28.0   

          kdist_country  kneighbors_country  label   latdiff   londiff  \
16046371            NaN                 NaN      0 -0.000876  0.000043   

          manhattan  ...  country_sim  country_gesh  country_leven  \
16046371   0.000919  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
16046371           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
16046371             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nl

...

(2284898, 91)
                        id          match_id     kdist  kneighbors  \
12960931  E_c2ee89f9e95918  E_1995a66258f613  0.020542        22.0   

          kdist_country  kneighbors_country  label   latdiff   londiff  \
12960931            NaN                 NaN      0 -0.018861 -0.008137   

          manhattan  ...  country_sim  country_gesh  country_leven  \
12960931   0.026999  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
12960931           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
12960931             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nl

  0%|          | 0/10 [00:00<?, ?it/s]

(2284998, 91)
                        id          match_id     kdist  kneighbors  \
17574524  E_dd20dfce1113a6  E_9376c77a4f6a63  0.043193        30.0   

          kdist_country  kneighbors_country  label   latdiff   londiff  \
17574524            NaN                 NaN      0  0.038441 -0.019696   

          manhattan  ...  country_sim  country_gesh  country_leven  \
17574524   0.058137  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
17574524           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
17574524             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nl

  0%|          | 0/10 [00:00<?, ?it/s]

(2285142, 91)
                        id          match_id     kdist  kneighbors  \
11934316  E_f57bc461f8b56a  E_a0c15304923c76  0.001313        20.0   

          kdist_country  kneighbors_country  label   latdiff   londiff  \
11934316            NaN                 NaN      0 -0.000943  0.000913   

          manhattan  ...  country_sim  country_gesh  country_leven  \
11934316   0.001856  ...          1.0           1.0            0.0   

          country_jaro  country_lcs  country_fuz  country_len_diff  \
11934316           1.0          2.0        100.0               0.0   

          country_nleven  country_nlcsk  country_nlcs  
11934316             0.0            1.0           1.0  

[1 rows x 91 columns]
Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_fuz', 'name_len_diff', 'name_nl

In [ ]:
!cp * /content/drive/MyDrive/datas/foursquare/traindata3